# Project: Finding Lines On the Road
- **Environment: python 3.6**

## Ideas of lane detection pipeline
 - cv2.inRange() for color selection
 - cv2.fillPoly() for regions selection
 - cv2.line() to draw lines on an image
 - cv2.addWeighted() to coadd/overlay two images
 - cv2.imwrite() to output images to file
 - cv2.itwise_and() to apply a mask to an image

## Import packages

In [114]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math

## Read image and convert to gray scale

In [115]:
def readImage(path):
    image = cv2.imread(path)
    return image

def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def drawImage(img):
    plt.imshow(img)

## Find edge

In [116]:
def gaussian_blur(img, kernal_size):
    return cv2.GaussianBlur(img, (kernal_size, kernal_size), 0)
    
def findEdge(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

## Find region of interests

In [117]:
def regionOfInterest(img, vertices):
    mask = np.zeros_like(img)
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

## Hough lines
 - lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),min_line_length, max_line_gap)
 - rho in units of pixels and theta in units of radians, rho takes a minimum value of 1, and a reasonable starting place for theta is 1 degree (pi/180 in radians).
 - the threshold parameter specifies the minimum number of votes (intersections in a given grid cell) 
 - the empty np.array([]) is just a placeholder, no need to change it.
 - min_line_length is the minimum length of a line (in pixels) that you will accept in the output, 
 - max_line_gap is the maximum distance (again, in pixels) between segments that you will allow to be connected into a single line. 


In [118]:
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    # Returns an image with hough lines drawn.
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

## Draw lines on image

In [119]:
def draw_lines(img, lines, color=[0, 0, 255], thickness=2):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

In [120]:
def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

## Save images

In [121]:
def saveImg(img, path):
    try:
        cv2.imwrite(path, img)
    except Exception as e:
        print('same image error',e)

## Test images
 - Build your pipeline that will draw lane lines on the test_images
 - Save them to the test_images_output directory.

In [122]:
import os
from tqdm import tqdm 
DIR = 'test_images'
for name in tqdm(os.listdir(DIR)):
    path = os.path.join(DIR, name)
    image = readImage(path)
    gray = grayscale(image)
    edge = findEdge(gaussian_blur(gray,3), 100, 200)
    # create vertices and roi image
    left_bottom = (0, image.shape[0])
    right_bottom = (image.shape[1], image.shape[0])
    left_top = (image.shape[1]/3, image.shape[0]*2/3)
    right_top = (image.shape[1]*2/3, image.shape[0]*2/3)
    vertices = np.array([[left_top, right_top, right_bottom, left_bottom]], dtype=np.int32)
    maskedImage = regionOfInterest(edge, vertices)
    
    # create hough lines
    rho = 1
    theta = np.pi/180
    threshold = 1
    min_line_length = 30
    max_line_gap = 10
    line_img = hough_lines(maskedImage, rho, theta, 
                        threshold, min_line_length, max_line_gap)
    
    # draw on image
    result = weighted_img(line_img, image)
    saveImg(result, f'test_image_output/{name}.jpg')

100%|██████████| 6/6 [00:00<00:00, 43.45it/s]
